# Joel Brigida
## CDA-4321: Cryptographic Engineering
## Public Key Cryptography Functions

### RSA Key Generation In Sage
- Enter Number of bits to generate that length key.

In [1]:
def rsa(bits):
    # pseudo-primality test if bits > 1024
    proof = (bits <= 1024)
    p = next_prime(ZZ.random_element(2**(bits // 2 + 1)), proof = proof)
    q = next_prime(ZZ.random_element(2**(bits // 2 + 1)), proof = proof)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    while True:
        e = ZZ.random_element(1, phi_n)
        if gcd(e, phi_n)==1: 
            break
    d = lift(Mod(e, phi_n)^(-1))
    return p, q, e, d, n        # p & q = primes, e = public key, d = private key, n = p * q

In [2]:
p1, q1, e1, d1, n1 = rsa(256)

print(f'p1 = {hex(p1)}\nq1 = {hex(q1)}\ne1 = {hex(e1)}\nd1 = {hex(d1)}\nn1 = {hex(n1)}\n')

p1 = 0xc4ef69e0947e2501c40a2944de22f11f
q1 = 0xef5c31028935afc8c85b91e5901b0df
e1 = 0x1c815230025aed030bcd22c37fae4b95e9954f18f46cf1437c3116e82b049bd
d1 = 0x106c3a68036bf30b3ade914af4cb33eebfd7d70618ea6e5b153550cc8db02d
n1 = 0xb8226f967e075e837a3c89a7be11d4959c79d5d80ba1ed55d2af84e3e545a01



### Key Generation using Mersenne Numbers:

In [3]:
p = (2^31) - 1
q = (2^61) - 1

n = p * q
phi = (p - 1) * (q - 1)

print(f'Phi = {phi}')

e = ZZ.random_element(phi)
while gcd(e, phi) != 1:
    e = ZZ.random_element(phi)

print(f'Random e = {e}')
print(f'Is e < n? {e < n}\n')

print(f'n = {n}')
print(f'e = {e}')
print(f'Phi = {phi}\n')

bezout = xgcd(e, phi)
print(f'bezout = {bezout}')

d = Integer(mod(bezout[1], phi))
print(f'd = {d}\n')

print(f'mod(d * e, phi) = {mod(d * e, phi)}')

Phi = 4951760152529835076874141700
Random e = 4324234782253644443530326589
Is e < n? True

n = 4951760154835678088235319297
e = 4324234782253644443530326589
Phi = 4951760152529835076874141700

bezout = (1, 1280103053126275167154041109, -1117878486979963222660764416)
d = 1280103053126275167154041109

mod(d * e, phi) = 1


### Encoding & Decoding A Message: 
- #### String $\Longleftrightarrow$ Integer

In [4]:
def encode(s):
    s = str(s)
    return sum(ord(s[i]) * 256^i for i in range(len(s)))

def decode(n):
    n = Integer(n)
    v = []
    while n!= 0:
        v.append(chr(n % 256))
        n //= 256
    return ''.join(v)

#### Encoding and Decoding Example:

In [5]:
myMessage = "Test Cryptography Message"
print(f'n1 == p1 * q1 ? {p1 * q1 == n1}\n')

plainTextBitPattern = encode(myMessage) # convert the string to an integer
m1 = plainTextBitPattern
print(f'Plain Text Message Bit Pattern: {hex(m1)}\n')

decodedMessage = decode(m1)             # convert unencrypted message to text as a test
print(f'Sanity Check: Unencrypted Message = \"{decodedMessage}\"')

cypherText = power_mod(m1, e1, n1)      # performs m^e mod n to encrypt message to cypher text
c1 = cypherText
print(f'c1 Encrypted Bit Pattern = {hex(c1)}')

n1 == p1 * q1 ? True

Plain Text Message Bit Pattern: 0x6567617373654d207968706172676f74707972432074736554

Sanity Check: Unencrypted Message = "Test Cryptography Message"
c1 Encrypted Bit Pattern = 0x92013e2c3159095894cdfc8bfef3f2dfaac273e57bffe3b627eb7850b8f98e4


### Encrypting A Bit Pattern:
- Given $(n, e)$ and $(n, d)$ as computed above
- To encrypt a bit pattern into a sequence of numbers $m$, compute: 
### $$ c = m^{e} \bmod n $$

In [6]:
m = 72697676798779827668        # Example random integer
print(f'm = {m}')
print(f'e = {e}')
print(f'n = {n}', '\n')

c = power_mod(m, e, n)
print(f'Encrypted Message c = {c}')

m = 72697676798779827668
e = 4324234782253644443530326589
n = 4951760154835678088235319297 

Encrypted Message c = 866690494698149907360211859


### Decrypt the CypherText Bit Pattern: $$ m = c^d \bmod n $$

In [7]:
m2 = power_mod(c1, d1, n1)
print(f'Decrypted Message \'m2\' Bit Pattern: {hex(m2)}\n')

newMessage = decode(m2)
print(f'Decrypted Message Text: \"{newMessage}\"')

Decrypted Message 'm2' Bit Pattern: 0x6567617373654d207968706172676f74707972432074736554

Decrypted Message Text: "Test Cryptography Message"


#### Testing XOR Logic Operations:

In [8]:
a = 0x1234
b = 0x4567
c = a ^^ b

print(f'a XOR b = {hex(c)} = {bin(c)}\n')

print(f'a XOR c = {hex(a ^^ c)} == b')
print(f'a XOR b = {hex(b ^^ c)} == c\n')

print(f'a XOR 0 = {hex(a ^^ 0)} == a')
print(f'b XOR 0 = {hex(b ^^ 0)} == b\n')

# Recall that (a ^^ b) ^^ a = b and (a ^^ b) ^^ b = a

# 16 bits in total:
print(hex(0b1111111111111000))

a XOR b = 0x5753 = 0b101011101010011

a XOR c = 0x4567 == b
a XOR b = 0x1234 == c

a XOR 0 = 0x1234 == a
b XOR 0 = 0x4567 == b

0xfff8
